## Read and clean data as needed

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap

In [ ]:
for dirname, _, filenames in os.walk('/Resources'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("Resources/data.csv")
df.head()

In [ ]:
# check dataset
df.describe()

In [ ]:
# check columns
df.columns

In [ ]:
# drop the last column
df = df.drop(columns = 'Unnamed: 32')

In [ ]:
# check data
df

## Preliminary Analysis (Descriptives/Statistics Summary)

In [ ]:
# drop id column for further analysis
updated_df = df.drop(columns='id')

# check dataset
updated_df.head()

In [ ]:
sns.set(style="darkgrid")

# Counts of benign and malignant tumors
ax_bar = sns.countplot(x="diagnosis", data=updated_df, palette="Set3")

In [ ]:
# recode diagnosis column to 1 (malignant) and 0 (benign)
def tumor(row):
    if row['diagnosis'] == 'B':
        return 0
    if row['diagnosis'] == 'M':
        return 1
    
# create a new column with the recoded values
updated_df['tumor'] = updated_df.apply (lambda row: tumor(row), axis=1)

# calculate correlation coefficients
corr_df = updated_df.corr()
corr_df.head()

In [ ]:
# correlation map
f,ax1 = plt.subplots(figsize=(18, 18))
sns.heatmap(updated_df.corr(), cmap='BuPu',annot=True, linewidths=.5, fmt= '.1f',ax=ax1)
plt.xticks(fontsize=11,rotation=70)
plt.show()

## Prep Data for Classifical Models

In [ ]:
# create the y variable
y= df['diagnosis'].map({'M':1,'B':0})

In [ ]:
# create a dataframe with selected features based on correlation results (keeping those with coefficient of .5 and above)
X = df[['radius_mean', 'perimeter_mean', 'area_mean',
       'compactness_mean', 'concavity_mean', 'concave points_mean',
       'radius_se', 'perimeter_se', 'area_se', 
       'radius_worst', 'texture_worst',
       'perimeter_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst']]

In [ ]:
# create train and test dataset

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, random_state=42)
N ,D = X_train.shape

In [ ]:
# scale/normalize the train and test data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
# This function runs the Predictor. It prints how many inaccurate predictions were made and whether the
# model is over predicting or under predicting
def Predictor(classifier, name):
    predictions = classifier.predict(X_test)
    predictions_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
    predictions_df["Sum"] = predictions_df.sum(axis=1)
    predictions_df = predictions_df[predictions_df.Sum == 1]

    inaccurate_predictions = len(predictions_df)
    false_positives = predictions_df["Prediction"].sum()
    false_negatives = predictions_df["Actual"].sum()
    difference = false_positives - false_negatives

    print(f"Results for {name} classifier:\n"
          f"Inaccurate Predictions: {inaccurate_predictions}\n"
          f"False Positives: {false_positives}\n"
          f"False Negatives: {false_negatives}\n"
          f"Difference (positive is good): {difference}\n")

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()

In [ ]:
classifier.fit(X_train, y_train)

## Support Vector Machines

## Decision Tree Algorithm

In [ ]:
from sklearn import tree

In [ ]:
feature_names = X.columns

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
Predictor(clf, "Decision Tree")

## Random Forest Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

In [ ]:
Predictor(rf, "Random Forest")

## K Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
print('k=3 Test Acc: %.3f' % knn.score(X_test, y_test))

In [ ]:
Predictor(knn, "K Nearest Neighbor")

## Running the Predictor on all models

In [ ]:
#Predictor(classifier, "Logistic Regression") <-- needs to be updated with correct classifier
#Predictor(classifier, "Support Vector Machines") <-- needs to be updated with correct classifier
Predictor(clf, "Decision Tree")
Predictor(rf, "Random Forest")
Predictor(knn, "K Nearest Neighbor")